In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from dataset import make_dataset
from os.path import join


train_path = join('jeju_data', 'train_new.parquet')
test_path = join('jeju_data', 'test_new.parquet')

x_train, y_train, test = make_dataset(train_path, test_path)

시작시간: 2022-10-17 17:08:17.103751
Train dataset success !
Test dataset success !
끝난 시간:  0:01:06.058098


In [1]:
X = x_train.copy()
y = y_train.copy()

# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=1103)

NameError: name 'x_train' is not defined

In [13]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from catboost import CatBoostRegressor

In [17]:

def objective(trial):
  param = {
      "random_state":42,
      'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
      'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
      "n_estimators":trial.suggest_int("n_estimators", 500, 5000),
      "max_depth":trial.suggest_int("max_depth", 4, 16),
      'random_strength' :trial.suggest_int('random_strength', 0, 100),
      "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
      "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
      "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
      "max_bin": trial.suggest_int("max_bin", 200, 500),
      'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
  }
  X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2)
  cat = CatBoostRegressor(**param)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=35,
          verbose=100)
  cat_pred = cat.predict(X_valid)
  mae = mean_absolute_error(y_valid, cat_pred)

  return mae

sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'minimize',
    sampler = sampler,
)

study.optimize(objective, n_trials=30)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-10-17 18:25:03,194] A new study created in memory with name: cat_parameter_opt
C:\Users\jkpark\AppData\Local\Temp\ipykernel_10844\1892245849.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
C:\Users\jkpark\AppData\Local\Temp\ipykernel_10844\1892245849.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),


0:	learn: 15.7623319	test: 15.7623319	test1: 15.7678888	best: 15.7678888 (0)	total: 408ms	remaining: 25m 48s
100:	learn: 8.4588914	test: 8.4588914	test1: 8.4749176	best: 8.4749176 (100)	total: 27.3s	remaining: 16m 38s
200:	learn: 7.3260463	test: 7.3260463	test1: 7.3448786	best: 7.3448786 (200)	total: 53.4s	remaining: 15m 55s
300:	learn: 6.8901356	test: 6.8901356	test1: 6.9100832	best: 6.9100832 (300)	total: 1m 19s	remaining: 15m 24s
400:	learn: 6.6497557	test: 6.6497557	test1: 6.6700284	best: 6.6700284 (400)	total: 1m 58s	remaining: 16m 40s
500:	learn: 6.4697511	test: 6.4697511	test1: 6.4901301	best: 6.4901301 (500)	total: 2m 44s	remaining: 18m
600:	learn: 6.3222749	test: 6.3222749	test1: 6.3428719	best: 6.3428719 (600)	total: 3m 30s	remaining: 18m 37s
700:	learn: 6.2029225	test: 6.2029225	test1: 6.2242532	best: 6.2242532 (700)	total: 4m 13s	remaining: 18m 38s
800:	learn: 6.0873073	test: 6.0873073	test1: 6.1088390	best: 6.1088390 (800)	total: 5m 1s	remaining: 18m 44s
900:	learn: 5.9723

[I 2022-10-17 18:51:47,288] Trial 0 finished with value: 3.417697037735075 and parameters: {'learning_rate': 0.018272261776066247, 'bagging_temperature': 63.512210106407046, 'n_estimators': 3794, 'max_depth': 11, 'random_strength': 15, 'colsample_bylevel': 0.49359671220172163, 'l2_leaf_reg': 1.7519275289243016e-06, 'min_child_samples': 88, 'max_bin': 380, 'od_type': 'IncToDec'}. Best is trial 0 with value: 3.417697037735075.
C:\Users\jkpark\AppData\Local\Temp\ipykernel_10844\1892245849.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
C:\Users\jkpark\AppData\Local\Temp\ipykernel_10844\1892245849.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/

0:	learn: 15.5418916	test: 15.5418916	test1: 15.5504199	best: 15.5504199 (0)	total: 215ms	remaining: 5m 12s
100:	learn: 8.5701553	test: 8.5701553	test1: 8.5590589	best: 8.5590589 (100)	total: 23.4s	remaining: 5m 13s
200:	learn: 7.7623657	test: 7.7623657	test1: 7.7510258	best: 7.7510258 (200)	total: 47.4s	remaining: 4m 55s
300:	learn: 7.3430810	test: 7.3430810	test1: 7.3325763	best: 7.3325763 (300)	total: 1m 12s	remaining: 4m 38s
400:	learn: 6.9717928	test: 6.9717928	test1: 6.9624760	best: 6.9624760 (400)	total: 1m 39s	remaining: 4m 20s
500:	learn: 6.6968717	test: 6.6968717	test1: 6.6876613	best: 6.6876613 (500)	total: 2m 5s	remaining: 3m 59s
600:	learn: 6.4888672	test: 6.4888672	test1: 6.4794328	best: 6.4794328 (600)	total: 2m 32s	remaining: 3m 36s
700:	learn: 6.3069457	test: 6.3069457	test1: 6.2984759	best: 6.2984759 (700)	total: 2m 57s	remaining: 3m 11s
800:	learn: 6.1806035	test: 6.1806035	test1: 6.1722869	best: 6.1722869 (800)	total: 3m 22s	remaining: 2m 45s
900:	learn: 6.0609403	t

[I 2022-10-17 18:57:56,124] Trial 1 finished with value: 4.164132959573354 and parameters: {'learning_rate': 0.04763628595029446, 'bagging_temperature': 21.368329072358772, 'n_estimators': 1455, 'max_depth': 6, 'random_strength': 18, 'colsample_bylevel': 0.5825453457757226, 'l2_leaf_reg': 1.5747445384650815e-05, 'min_child_samples': 46, 'max_bin': 287, 'od_type': 'IncToDec'}. Best is trial 0 with value: 3.417697037735075.
C:\Users\jkpark\AppData\Local\Temp\ipykernel_10844\1892245849.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
C:\Users\jkpark\AppData\Local\Temp\ipykernel_10844\1892245849.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/rel

...

0:	learn: 15.4755908	test: 15.4755908	test1: 15.4628053	best: 15.4628053 (0)	total: 1.26s	remaining: 1h 20m 24s
100:	learn: 7.3548948	test: 7.3548948	test1: 7.3562359	best: 7.3562359 (100)	total: 1m 25s	remaining: 52m 18s
200:	learn: 6.7279016	test: 6.7279016	test1: 6.7338393	best: 6.7338393 (200)	total: 2m 37s	remaining: 47m 22s
300:	learn: 6.4750820	test: 6.4750820	test1: 6.4844649	best: 6.4844649 (300)	total: 3m 34s	remaining: 41m 51s
400:	learn: 5.9018485	test: 5.9018485	test1: 5.9113922	best: 5.9113922 (400)	total: 5m 13s	remaining: 44m 43s
500:	learn: 5.3942023	test: 5.3942023	test1: 5.4034589	best: 5.4034589 (500)	total: 7m 6s	remaining: 47m 13s
600:	learn: 5.1143016	test: 5.1143016	test1: 5.1246569	best: 5.1246569 (600)	total: 8m 56s	remaining: 48m 2s
700:	learn: 4.9344121	test: 4.9344121	test1: 4.9471988	best: 4.9471988 (700)	total: 10m 48s	remaining: 48m 15s
800:	learn: 4.8079099	test: 4.8079099	test1: 4.8260825	best: 4.8260825 (800)	total: 12m 43s	remaining: 48m 5s
900:	lear

[I 2022-10-18 13:05:00,771] Trial 28 finished with value: 2.985363992094312 and parameters: {'learning_rate': 0.047360814959130804, 'bagging_temperature': 40.52044595178952, 'n_estimators': 3830, 'max_depth': 16, 'random_strength': 86, 'colsample_bylevel': 0.4129957917974832, 'l2_leaf_reg': 1.7654281892238895e-05, 'min_child_samples': 24, 'max_bin': 378, 'od_type': 'Iter'}. Best is trial 28 with value: 2.985363992094312.
C:\Users\jkpark\AppData\Local\Temp\ipykernel_10844\1892245849.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
C:\Users\jkpark\AppData\Local\Temp\ipykernel_10844\1892245849.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/rele

0:	learn: 15.6628576	test: 15.6628576	test1: 15.6800071	best: 15.6800071 (0)	total: 261ms	remaining: 16m 22s
100:	learn: 7.4192137	test: 7.4192137	test1: 7.4296747	best: 7.4296747 (100)	total: 1m 28s	remaining: 53m 21s
200:	learn: 6.8135565	test: 6.8135565	test1: 6.8268311	best: 6.8268311 (200)	total: 2m 50s	remaining: 50m 26s
300:	learn: 6.5197870	test: 6.5197870	test1: 6.5343567	best: 6.5343567 (300)	total: 4m 5s	remaining: 47m 7s
400:	learn: 5.9227020	test: 5.9227020	test1: 5.9396429	best: 5.9396429 (400)	total: 6m 18s	remaining: 52m 54s
500:	learn: 5.4453539	test: 5.4453539	test1: 5.4649933	best: 5.4649933 (500)	total: 8m 18s	remaining: 54m 11s
600:	learn: 5.1822945	test: 5.1822945	test1: 5.2045116	best: 5.2045116 (600)	total: 10m 15s	remaining: 54m 2s
700:	learn: 4.9973032	test: 4.9973032	test1: 5.0225251	best: 5.0225251 (700)	total: 12m 17s	remaining: 53m 46s
800:	learn: 4.8733448	test: 4.8733448	test1: 4.9023387	best: 4.9023387 (800)	total: 14m 19s	remaining: 53m 4s
900:	learn: 

[I 2022-10-18 14:24:49,689] Trial 29 finished with value: 3.021950775381675 and parameters: {'learning_rate': 0.04875709506907699, 'bagging_temperature': 49.87581533270841, 'n_estimators': 3768, 'max_depth': 15, 'random_strength': 90, 'colsample_bylevel': 0.4492431292043895, 'l2_leaf_reg': 1.6032202541045213e-05, 'min_child_samples': 25, 'max_bin': 368, 'od_type': 'Iter'}. Best is trial 28 with value: 2.985363992094312.


Best Score: 2.985363992094312
Best trial {'learning_rate': 0.047360814959130804, 'bagging_temperature': 40.52044595178952, 'n_estimators': 3830, 'max_depth': 16, 'random_strength': 86, 'colsample_bylevel': 0.4129957917974832, 'l2_leaf_reg': 1.7654281892238895e-05, 'min_child_samples': 24, 'max_bin': 378, 'od_type': 'Iter'}


Best trial {'learning_rate': 0.047360814959130804, 'bagging_temperature': 40.52044595178952, 'n_estimators': 3830, 'max_depth': 16, 'random_strength': 86, 'colsample_bylevel': 0.4129957917974832, 'l2_leaf_reg': 1.7654281892238895e-05, 'min_child_samples': 24, 'max_bin': 378, 'od_type': 'Iter'}


In [18]:
params = {'learning_rate': 0.047360814959130804, 'bagging_temperature': 40.52044595178952, 'n_estimators': 3830, 'max_depth': 16, 'random_strength': 86, 'colsample_bylevel': 0.4129957917974832, 'l2_leaf_reg': 1.7654281892238895e-05, 'min_child_samples': 24, 'max_bin': 378, 'od_type': 'Iter'}
cat_model = CatBoostRegressor(**params).fit(X, y)
y_pred = cat_model.predict(test)

import pandas as pd

sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')
sample_submission['target'] = y_pred
sample_submission.to_csv("./submit_cat_optuna.csv", index=False)

0:	learn: 15.5840963	total: 217ms	remaining: 13m 50s
1:	learn: 15.1641900	total: 2.02s	remaining: 1h 4m 26s
2:	learn: 14.7377321	total: 3.88s	remaining: 1h 22m 28s
3:	learn: 14.3956236	total: 4.73s	remaining: 1h 15m 28s
4:	learn: 14.3006864	total: 4.88s	remaining: 1h 2m 10s
5:	learn: 13.9590013	total: 6.61s	remaining: 1h 10m 9s
6:	learn: 13.6290986	total: 7.21s	remaining: 1h 5m 37s
7:	learn: 13.3228464	total: 7.68s	remaining: 1h 1m 9s
8:	learn: 13.0195834	total: 8.79s	remaining: 1h 2m 12s
9:	learn: 12.7137919	total: 10.5s	remaining: 1h 6m 42s
10:	learn: 12.4213183	total: 11.3s	remaining: 1h 5m 34s
11:	learn: 12.1305985	total: 11.9s	remaining: 1h 3m 14s
12:	learn: 11.8691944	total: 13.4s	remaining: 1h 5m 34s
13:	learn: 11.6508492	total: 13.9s	remaining: 1h 3m 15s
14:	learn: 11.4083189	total: 15.3s	remaining: 1h 4m 50s
15:	learn: 11.2028539	total: 16.6s	remaining: 1h 5m 57s
16:	learn: 11.0043435	total: 17.9s	remaining: 1h 7m 3s
17:	learn: 10.8301234	total: 19.2s	remaining: 1h 7m 56s
18:	

In [25]:
a = pd.read_csv('submit_cat_optuna.csv')
b = pd.read_csv('submit_xgb_optuna.csv')
c = (a['target']*0.5) + (b['target']*0.5)
sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')
sample_submission['target'] = c
sample_submission.to_csv("./submit.csv", index=False)